In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
torch.cuda.is_available()

False

In [3]:
if torch.cuda.is_available():
    torch.cuda.get_device_name(0)
else:
    print("No GPU")

No GPU


# **Part 1 - Loading the Data**

In [5]:
#Opening Pride & Prejudice 
with open('pride_and_prejudice.txt', 'r', encoding = 'utf8') as f:
#code 'r' stands for reading, encoding 'utf8' passes in the file as a string, and save this all as variable, f
  text = f.read()

In [7]:
text[:1000] #\n stands for new line

'Chapter 1\n\n\nIt is a truth universally acknowledged, that a single man in possession\nof a good fortune, must be in want of a wife.\n\nHowever little known the feelings or views of such a man may be on his\nfirst entering a neighbourhood, this truth is so well fixed in the minds\nof the surrounding families, that he is considered the rightful property\nof some one or other of their daughters.\n\n“My dear Mr. Bennet,” said his lady to him one day, “have you heard that\nNetherfield Park is let at last?”\n\nMr. Bennet replied that he had not.\n\n“But it is,” returned she; “for Mrs. Long has just been here, and she\ntold me all about it.”\n\nMr. Bennet made no answer.\n\n“Do you not want to know who has taken it?” cried his wife impatiently.\n\n“_You_ want to tell me, and I have no objection to hearing it.”\n\nThis was invitation enough.\n\n“Why, my dear, you must know, Mrs. Long says that Netherfield is taken\nby a young man of large fortune from the north of England; that he came\ndow

In [9]:
len(text) #so we got 400k characters

684743

# **Part 2 - Encoding the Data**

In [10]:
all_characters = set(text) #now we're separating each individual character in this text set
print(all_characters) #it returned all the unique characters
print(len(all_characters)) #84 unique characters

{'L', '6', 'x', 'k', 'p', 'M', 'C', 'z', 'n', 'E', '”', 'Y', '!', 'F', '(', '.', 'D', 'e', 'N', 'H', 'R', '4', '8', 'd', '?', 'Z', 'r', "'", 'O', 't', 'W', '\n', 'S', 'G', ':', ',', 'b', 'f', 'A', 'm', 'J', 'V', '3', '7', 'y', 'g', '*', '0', '1', 's', 'K', 'i', '9', 'o', 'h', 'v', 'w', '5', '2', ';', '“', 'u', 'T', 'B', '_', ')', 'q', 'c', 'U', ' ', '-', 'j', 'a', 'I', 'l', 'P'}
76


In [11]:
#So we're going to create 2 objects, an encoder and de-coder
#An encoder will take a character/object and return it's encoded value
#A decoder will take a value, and return it's corresponding character/object
decoder = dict(enumerate(all_characters))#So all we did was create a dictionary by enumerating all the characters to get number value -- > characer
print(decoder) #see the dictionary now has a corresponding value for each character

{0: 'L', 1: '6', 2: 'x', 3: 'k', 4: 'p', 5: 'M', 6: 'C', 7: 'z', 8: 'n', 9: 'E', 10: '”', 11: 'Y', 12: '!', 13: 'F', 14: '(', 15: '.', 16: 'D', 17: 'e', 18: 'N', 19: 'H', 20: 'R', 21: '4', 22: '8', 23: 'd', 24: '?', 25: 'Z', 26: 'r', 27: "'", 28: 'O', 29: 't', 30: 'W', 31: '\n', 32: 'S', 33: 'G', 34: ':', 35: ',', 36: 'b', 37: 'f', 38: 'A', 39: 'm', 40: 'J', 41: 'V', 42: '3', 43: '7', 44: 'y', 45: 'g', 46: '*', 47: '0', 48: '1', 49: 's', 50: 'K', 51: 'i', 52: '9', 53: 'o', 54: 'h', 55: 'v', 56: 'w', 57: '5', 58: '2', 59: ';', 60: '“', 61: 'u', 62: 'T', 63: 'B', 64: '_', 65: ')', 66: 'q', 67: 'c', 68: 'U', 69: ' ', 70: '-', 71: 'j', 72: 'a', 73: 'I', 74: 'l', 75: 'P'}


In [12]:
encoder = {char: ind for ind, char in decoder.items()} #for index and character in decoder, since each dictionary item is an index + character
print(encoder) #notice the values match up

{'L': 0, '6': 1, 'x': 2, 'k': 3, 'p': 4, 'M': 5, 'C': 6, 'z': 7, 'n': 8, 'E': 9, '”': 10, 'Y': 11, '!': 12, 'F': 13, '(': 14, '.': 15, 'D': 16, 'e': 17, 'N': 18, 'H': 19, 'R': 20, '4': 21, '8': 22, 'd': 23, '?': 24, 'Z': 25, 'r': 26, "'": 27, 'O': 28, 't': 29, 'W': 30, '\n': 31, 'S': 32, 'G': 33, ':': 34, ',': 35, 'b': 36, 'f': 37, 'A': 38, 'm': 39, 'J': 40, 'V': 41, '3': 42, '7': 43, 'y': 44, 'g': 45, '*': 46, '0': 47, '1': 48, 's': 49, 'K': 50, 'i': 51, '9': 52, 'o': 53, 'h': 54, 'v': 55, 'w': 56, '5': 57, '2': 58, ';': 59, '“': 60, 'u': 61, 'T': 62, 'B': 63, '_': 64, ')': 65, 'q': 66, 'c': 67, 'U': 68, ' ': 69, '-': 70, 'j': 71, 'a': 72, 'I': 73, 'l': 74, 'P': 75}


In [13]:
encoded_text = np.array([encoder[char] for char in text]) #creating a numpy array for each character in text with it's corresponding encoded value
encoded_text[:200] #notice how each character is replaced by its corresponding encoded value now

array([ 6, 54, 72,  4, 29, 17, 26, 69, 48, 31, 31, 31, 73, 29, 69, 51, 49,
       69, 72, 69, 29, 26, 61, 29, 54, 69, 61,  8, 51, 55, 17, 26, 49, 72,
       74, 74, 44, 69, 72, 67,  3,  8, 53, 56, 74, 17, 23, 45, 17, 23, 35,
       69, 29, 54, 72, 29, 69, 72, 69, 49, 51,  8, 45, 74, 17, 69, 39, 72,
        8, 69, 51,  8, 69,  4, 53, 49, 49, 17, 49, 49, 51, 53,  8, 31, 53,
       37, 69, 72, 69, 45, 53, 53, 23, 69, 37, 53, 26, 29, 61,  8, 17, 35,
       69, 39, 61, 49, 29, 69, 36, 17, 69, 51,  8, 69, 56, 72,  8, 29, 69,
       53, 37, 69, 72, 69, 56, 51, 37, 17, 15, 31, 31, 19, 53, 56, 17, 55,
       17, 26, 69, 74, 51, 29, 29, 74, 17, 69,  3,  8, 53, 56,  8, 69, 29,
       54, 17, 69, 37, 17, 17, 74, 51,  8, 45, 49, 69, 53, 26, 69, 55, 51,
       17, 56, 49, 69, 53, 37, 69, 49, 61, 67, 54, 69, 72, 69, 39, 72,  8,
       69, 39, 72, 44, 69, 36, 17, 69, 53,  8, 69, 54, 51])

In [14]:
decoder[19] #see how 19 = H based on our dictionary 

'H'

In [15]:
#Now applying a one hot encoding to the data
def one_hot_encoder(encoded_text, num_unique_chars):
  #encoded_text will be a batch of encoded text, so not all of the text data at once
  #num_unique_chars is len(set(text)), which is the sum of how many unique characters that we have. But this is for the batch of text, unlike what we did above for all the data at once
  one_hot = np.zeros((encoded_text.size, num_unique_chars)) #intiailizing our one hot encoding matrix with all zeros and dimensions size of encoded data by num of unique characters
  one_hot = one_hot.astype(np.float32) #making sure it's a float type

  one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0 #so here we're basically taking the shape of one_hot and flattening the encoded text, and where-ever the index position aligns, set it to 1. So basically 'L' = 0, will now be [1,0,0..0] and 'x' = 2, will now be [0,0,1..0]
  #read this https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array

  one_hot = one_hot.reshape((*encoded_text.shape,num_unique_chars)) #so reshaping our one hot to match the actual batch shape

  return one_hot

In [16]:
#testing out our function with an example array
example_arr = np.array([1,2,0])
one_hot_encoder(example_arr, 3) #see how they're all encoded now

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

# **Part 3 - Generating Training Batches**

In [17]:
example_text = np.arange(10)
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
example_text.reshape((5,-1)) #reshaping into samples per batch and 2 columns inferred

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [19]:
#Creating a function to generate batches
def generate_batches(encoded_text, samples_per_batch=10, seq_len=50):
  # X: encoded text of length seq_len
  # X_example = ([0,1,2],
  #              [1,2,3]) so we have 2 samples per batch with sequence length of 3 (per batch). Giving us 6 char_per_batch 


  # Y: encoded text shifted by one
  # Y_example = ([1,2,3],
  #              [2,3,4])
  char_per_batch = samples_per_batch * seq_len #total number of characters being used per batch

  num_batches_avail = int(len(encoded_text)/char_per_batch) #so finding how many batches tota by dividing the total length of encoded text by the number of characters per batch

  encoded_text = encoded_text[:num_batches_avail*char_per_batch] #so now we're just cutting off the end of the encoded text that won't fit evenly into a batch

  encoded_text = encoded_text.reshape((samples_per_batch, -1)) #now just reshaping it to rows = samples per batch, columns inferred to be the seq_len like we did with example text above
  #if encoded_text length is 40, and samples per batch is 2 and seq length of 4, then this will be shape (2,4) for now

  for n in range(0,encoded_text.shape[1], seq_len):
    #starting from 0 to column size of encoded_text with a step size of seq_len
    x = encoded_text[:,n:n+seq_len] #grab all the samples per batch (rows) at column n to n+seq_len (i.e. 0 to 4 if seq len is 4)
    y = np.zeros_like(x) #creating a zeros array with same shape as x

    try:
      y[:,:-1] = x[:,1:] #so we're making every row and every column till the last in y basically equal every sample in X starting from the second character till the end
      #y[1,2,3] = x[0,1,2,3]
      y[:,-1] = encoded_text[:, n+seq_len] #now we're assigning every row and the last column in y equal to n+seq_len, which will just be shifted over by 1
      #y[1,2,3,4] = encoded_text[0+4 = 4]
    except:
      #this is incase we get an out of range indexing error for the very end of our encoded text
      y[:,:-1] = x[:,1:]
      y[:,-1] = encoded_text[:,0]

    yield x,y

In [20]:
sample_text = encoded_text[:40]
sample_text
#len(sample_text)

array([ 6, 54, 72,  4, 29, 17, 26, 69, 48, 31, 31, 31, 73, 29, 69, 51, 49,
       69, 72, 69, 29, 26, 61, 29, 54, 69, 61,  8, 51, 55, 17, 26, 49, 72,
       74, 74, 44, 69, 72, 67])

In [21]:
batch_generator = generate_batches(sample_text, samples_per_batch = 2, seq_len=4)

x, y = next(batch_generator)
x #so made 2 samples (rows) with a seq len of 4 (columns), but also we cut the remaining 16 characters are since they didn't feed into our size
#the second row, [29,26,61,29] starts at character 21, since encoded text length is 40 and 2 samples per batch means each sample can be up to 20 characters

array([[ 6, 54, 72,  4],
       [29, 26, 61, 29]])

In [22]:
y #notice it's shifted over by 1

array([[54, 72,  4, 29],
       [26, 61, 29, 54]])

# **Part 4: Creating our LSTM Model**

In [23]:
class LSTM(nn.Module):
  
  def __init__(self, all_chars, num_hidden = 256, num_layers = 4, dropout_prob = 0.5, use_gpu = False):

    super().__init__() #super is a function that allows us to use Module's functions and variables, while also optimizing this inheritance process
    
    #Just intializing our attributes
    self.dropout_prob = dropout_prob
    self.num_hidden = num_hidden
    self.num_layers = num_layers
    self.use_gpu = use_gpu
    self.all_chars = all_chars
    self.decoder = dict(enumerate(all_chars))#So all we did was create a dictionary by enumerating all the characters to get number value assigned to each character
    self.encoder = {char: ind for ind, char in decoder.items()} #for index and character in decoder, since each dictionary item is an index + character

    #Adding LSTM layer
    self.lstm = nn.LSTM(input_size = len(self.all_chars), hidden_size = num_hidden, num_layers = num_layers, dropout = dropout_prob, batch_first = True,) #so here we're creating an LSTM where input size is one feature per unique character. Dropout here is for LSTM specifically
    self.dropout = nn.Dropout(dropout_prob) #drop out here is to go into the FC1
    #Adding fully connected layer
    self.fc_linear = nn.Linear(in_features = num_hidden, out_features = len(self.all_chars)) #now we're creating a fully connected layer that takes the hidden LSTM layers and can output to all unique characters


  def forward(self, x, hidden):
    lstm_out , hidden = self.lstm(x, hidden) #so our lstm expects inputs as: input, (h_0, c_0) = (seq_len, batch, input_size), (h_0, c_0)
    #so our first tuple contains len of the sequence, batch size, and input size. second tuple contains the hidden state and cell state
    #lstm outputs: output, (h_n, c_n) = (seq_len, batch, num_directions * hidden_size), (h_n, c_n), so we're doing tuple unpacking to update our hidden_state and creating a new variable for the for the output
    drop_output = self.dropout(lstm_out)
    drop_output = drop_output.contiguous().view(-1,self.num_hidden) #reshaping this dropout layer so that we can connect it to our output layer

    final_out = self.fc_linear(drop_output)
    return final_out, hidden

  def hidden_state(self, batch_size):
    if self.use_gpu:
      #Initializing h0 and c0 for cuda
      hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda(),
                torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda())
                #so recall the LSTM takes in 3 inputs: x(t), h(t-1), c(t-1), so we're passing 3 tensors into the LSTM layer
                #we are using zeros to basically initialize this hidden state, h(t-1), and cell state, c(t-1)
                #so we're creating a num_layers x batch_size x num_hidden sized matrix with only zeros
    else:
      #Initializing h0 and c0 for CPU
      hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden),
                torch.zeros(self.num_layers, batch_size, self.num_hidden))
    return hidden

In [30]:
model = LSTM(all_characters, num_hidden = 300, num_layers = 2, dropout_prob = 0.5, use_gpu = False)

In [31]:
total_param = []

for p in model.parameters():
  total_param.append(int(p.numel()))

sum(total_param) #so notice how the number of parameters is very close to the total number of characters. This is a good way to start adjusting the model parameters so that we can avoid overfitting (too many parameters) or underfitting (too few parameters)

1198876

In [32]:
criterion = nn.CrossEntropyLoss() #so our loss measurement will be Cross Entropy Loss since this is a categorical problem

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) #model parameters are just the fully connected layers and we are using Adam optimizer to optimize them
model.parameters #can see the parameters are just the fully connected layers and we are optimizing them

<bound method Module.parameters of LSTM(
  (lstm): LSTM(76, 300, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=300, out_features=76, bias=True)
)>

In [33]:
train_percent = 0.9 #use 90% of our data for training
train_index = int(len(encoded_text) * train_percent) #making our training index 90% of the length of the encoded text

train_data = encoded_text[:train_index] #so make our training data all the data from the start till the train_index in encoded text
val_data = encoded_text[train_index:] #making our validation from the train_index to the end

# **Part 5 - Training our Model**

In [34]:
import time #Gonna try to keep track of how long it takes to train our model

start_time = time.time()

epochs = 40
batch_size = 100
seq_len = 100
tracker = 0
num_char = max(encoded_text)+1

model.train()

if model.use_gpu:
  model.cuda()

#This for loop trains our NN
for i in range(epochs):
  #Forward propagation through our ANN using training data
  hidden = model.hidden_state(batch_size)

  for x,y in generate_batches(encoded_text = train_data, samples_per_batch = batch_size, seq_len = seq_len):
    tracker += 1
    #recall generate_batches returns x (training) and y (label)
    x = one_hot_encoder(encoded_text = x, num_unique_chars = num_char) #applying one hot encoding to our training data
    
    inputs = torch.from_numpy(x) #converting training data into a torch tensor
    targets = torch.from_numpy(y) #converting labels into a torch tensor

    if model.use_gpu:
      inputs = inputs.cuda()
      targets = targets.cuda()
    
    hidden = tuple([state.data for state in hidden]) #here we're grabbing the hidden state, so that we can reset it in the next line (otherwise it would backpropagate through all the training history)
    model.zero_grad() #resetting the gradient on the model so the hidden state doesn't accumulate

    lstm_output, hidden = model.forward(inputs, hidden) #recall our forward returns the final output and hidden

    #Calculating loss/error
    loss = criterion(lstm_output, targets.view(batch_size*seq_len).long()) #reshaping our targets into the correct format to compare to our predictions, lstm outputs 

    #Backpropagation
    loss.backward() #doing backpropagation off the loss function

    #Clipping off to avoid exploding gradient problem
    nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5) #capping it out at 5 to alleviate exploding gradient
    optimizer.step() #using the optimizer for the back propagation

    #tracking validation data
    if tracker % 50 == 0:
      val_hidden = model.hidden_state(batch_size)
      val_losses = []
      model.eval()

      for x,y in generate_batches(encoded_text = val_data, samples_per_batch =  batch_size, seq_len = seq_len): 
        #recall generate_batches returns x (training) and y (label)
        x = one_hot_encoder(encoded_text = x, num_unique_chars = num_char) #applying one hot encoding to our training data
        
        inputs = torch.from_numpy(x) #converting training data into a torch tensor
        targets = torch.from_numpy(y) #converting labels into a torch tensor

        if model.use_gpu:
          inputs = inputs.cuda()
          targets = targets.cuda()

        val_hidden = tuple([state.data for state in val_hidden]) #here we're resetting the hidden state, otherwise it would backpropagate through all the training history
        lstm_output, val_hidden = model.forward(inputs, val_hidden) #recall our forward returns the final output and hidden
        val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long()) #reshaping our targets into the correct format to compare to our predictions, lstm outputs 
        val_losses.append(val_loss.item())

      model.train()
      print(f'Epoch: {i}    Step: {tracker}     Val Loss: {val_loss.item():10.8f}')

print(f'Training took {(time.time() - start_time)/60} minutes')

Epoch: 0    Step: 50     Val Loss: 3.12415481
Epoch: 1    Step: 100     Val Loss: 3.10654974
Epoch: 2    Step: 150     Val Loss: 2.93200564
Epoch: 3    Step: 200     Val Loss: 2.64234090
Epoch: 4    Step: 250     Val Loss: 2.51444983
Epoch: 4    Step: 300     Val Loss: 2.41361380
Epoch: 5    Step: 350     Val Loss: 2.31937313
Epoch: 6    Step: 400     Val Loss: 2.24604893
Epoch: 7    Step: 450     Val Loss: 2.16895008
Epoch: 8    Step: 500     Val Loss: 2.10609818
Epoch: 9    Step: 550     Val Loss: 2.04523754
Epoch: 9    Step: 600     Val Loss: 1.99152970
Epoch: 10    Step: 650     Val Loss: 1.94423401
Epoch: 11    Step: 700     Val Loss: 1.90591168
Epoch: 12    Step: 750     Val Loss: 1.86582065
Epoch: 13    Step: 800     Val Loss: 1.83111167
Epoch: 13    Step: 850     Val Loss: 1.78864145
Epoch: 14    Step: 900     Val Loss: 1.76076841
Epoch: 15    Step: 950     Val Loss: 1.72695374
Epoch: 16    Step: 1000     Val Loss: 1.70063770
Epoch: 17    Step: 1050     Val Loss: 1.67548692
Epo

In [35]:
model_name = 'hidden300_layers2_Pride.net'
torch.save(model.state_dict(),model_name)

# **Part 6 - Generating Predictions**

In [37]:
#Making a function to predict the next character
def predict_next_character(model, char, hidden = None, k = 1):
  encoded_text = model.encoder[char]

  encoded_text = np.array([[encoded_text]]) #converting it into a numpy array

  encoded_text = one_hot_encoder(encoded_text = encoded_text, num_unique_chars = len(model.all_chars)) #one hot encoding our values

  inputs = torch.from_numpy(encoded_text) #converting our one hot encoding numpy array into a PyTorch tensor

  if (model.use_gpu):
    inputs = inputs.cuda() #use cuda on the inputs if we're using GPU

  hidden = tuple([state.data for state in hidden]) #here we're grabbing the hidden state

  lstm_out, hidden = model.forward(inputs, hidden) #Now we're running our model. recall our forward returns the final output and hidden

  probs = F.softmax(lstm_out, dim=1).data

  if (model.use_gpu):
    probs = probs.cpu() #moving the probabilities back to CPU so that we can use them with numpy

  # k determines how many characters to consider for our probability choice.
  # https://pytorch.org/docs/stable/torch.html#torch.topk
  # Return k largest probabilities in tensor
  probs, index_positions = probs.topk(k) #tuple unpacking to get the probabilty and index position of the character
        
  index_positions = index_positions.numpy().squeeze() #This is why we moved back to CPU above, so that we can convert index positions into a numpy array. We did .squeeze() to get it into the right shape. np.squeeze() removes axes of size 1 (singletons).
  #https://stackoverflow.com/questions/57237352/what-does-unsqueeze-do-in-pytorch
  #https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html

  probs = probs.numpy().flatten() #flattening into a single vector array so that we can do calculations on it in the next line

  probs = probs/probs.sum() 

  char = np.random.choice(index_positions, p=probs) #now selecting a random character of index_position and probability

  return model.decoder[char], hidden

In [38]:
def generate_text(model, size, seed = 'The', k = 1):
  #k=1 means grab the top 1 most probable character. Size is how many characters we want the model to predict. Seed means we're beginning with the word 'The'.
  if(model.use_gpu):
    model.cuda()
  else:
    model.cpu()
  
  model.eval()
  output_chars = [c for c in seed] #adding 'The' from seed, and will continue to append later for future characters predicted

  hidden = model.hidden_state(batch_size = 1) #batch size cuz we feeding in 1 character at a time

  for char in seed:
    char, hidden = predict_next_character(model = model, char = char, hidden = hidden, k = k)

  output_chars.append(char) #appending the characters that were decoded in predict_next_character

  for i in range(size):
    char, hidden = predict_next_character(model = model, char = output_chars[-1], hidden = hidden, k = k) #so now we're predicting characters based off the latest output_chars which was appended
    output_chars.append(char)

  return ''.join(output_chars) #so we're just adding the new output character

**Pride & Prejudice Text Generation Results**

In [39]:
print(generate_text(model = model, size = 1000,seed='The', k = 3)) #so taking the top 3 most likely choices for the next character

The she had been
all a month. Her side, as they had been the some of her and a sering, and his and she had a most acting to be their
presert of
this way on an endeave to be surprised to
the sister,” said her sister and she was somethand of all the reason of the country, and she was somether,
and her four and
heard
was
a marry as the place of any
other say which he had not to be some on her assurion which he sat she could have been all the reason to his sister, and taken the sonter, that they walked to
the motthe when they had not a sone that the
present of her
fair, and she was something of her acting a meater to be such as to brother, with all this so seemed as to
be such a most some acceptance, and that she can have a latter of her sister, who had been a little assured her at a much of thim to be sure her fate will be thein astentions of the party. She was not belees that they was some acquainted as tell that she had no manner of a last sister and she had always and thanking to him. 

**Shakespeare Text Generation Results**

In [ ]:
print(generate_text(model = model, size = 1000,seed='The', k = 3)) #copied and pasted from Shakespeare results

Ther shall not
  And take his senseless fingers. What, are you?
  OTHELLO. And what is that?
  OTHELLO. It is the wind of mark himself a man,
    As with the wint of such dispatch and things.
    This is the world, and some such serpent strikes
    The world to tender.
  MONTANO. What, will you take him on,
    Then that your sovereign lady? I had both  
    This world the season without many treasons
    And seem to strain how the stars of the storm
    Whose third importunate and subjects' seas
    They would have seen to hardly both of me.
    To take him with the choice, and the fair lords.
    We will be sent to send the world of heaven,
    The stones of heart that with his brain and strength
    They wished with a state. Though they would seem
    And wishes with a first as starved with him;
    And with a firm to see the shame thou hadst,
    Wherein I would have stood out of the court,
    And this, the secret shame, and that which shall
    Where strike away the foulter of hi

**Tom Sawyer Text Generation Results**

In [42]:
print(generate_text(model = model, size = 1000,seed='The', k = 3)) #copied and pasted from Tom Sawyer results

The wishing a courte to he word. The beat till a streat on his some and the sand an it wonthing to the book the say and hass and to the sunters and they sone and had there was a come of him and her and then anythen the
boy that this than treit and that way to
the cound of a porenters, and she say has the sare they was a suppinted of
her the hard to hear and
town an all her and that time and his him to him as a chomess and hers of the boys, and him his hadner of
the sured and
the bears, and his his had said a strate of he with had and strong and then a shound a courted himself that the conden to the
boys, and
his say a pontles and here thit had some his wanted to stear than the shoring to
had to the sand an the stracked to time the sardent and his winged a cross, and soment and to the sand, and soming.
He his was that he would the strown the came and the crosting of a coull the sain of the his aroust as the said:

“Oh, I'd seath on,”
-they handsed a than the condient to to the said and


**War and Peace Text Generation Results**

In [35]:
print(generate_text(model = model, size = 1000,seed='The', k = 3)) #copied and pasted from War & Peace results

There the same son’s army, as if he were still a moment of the carts of the countess,
who had seen all the cheets that the conversation, as to the passage of his soul,
before the countess, to the countess and a princess the
passage.

The storing of
her hand, and the captain went out of the
same son of that sound of the position, and a propose to the commander in chief of her
son, the sound of the country. The prince had been too seeing them the
sound of the campaign to to see the strange command of their presentes of horses at the soldious. The campaign. That he was so asking that the partical walking their honor of the same army. The conscalions, the partious and his hand, as he was so and that it was not a marriage that they was a contempt of
the country as the strangers at the strange tones, the same time was still all all the property, the count and him to the countess, and the packet was a serfs to her any the position, was the part of that countess, and having some time at
him.



**Meditation Text Generation Results**

In [34]:
print(generate_text(model = model, size = 1000,seed='The', k = 3)) #copied and pasted from Meditation results

The was trough of that
is a porse to bo that
is the wart and to the ware and a thing, thou dost not they shall thou may they are to the power things is it, as it thou dott neater and all to may the partion and thee and as the sane of in tree and such and sees, as the soun that which is a proses and to
the part of and any to to
that they shall as it to bo to be neither in the
seese and somming antt theme of a man shill and,
what as it is that what as it in the proper of in the pars on intended of the preselves and some, that whan thou deed in that is all that thou mane as it all time and all thas are subsion and to man to things ale as the panticire to to be neather of the world of the pract of
the seant of a mitted the same of that are such and
all things and thee wholly
that who that the soul of in the same the part and that that is is intind of to the world the sumpensest
and such of that wish any as in any things, and to theme any things it an one and to the would that whole the pro